## 필요한 라이브러리 import

In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os
import joblib

## 데이터 준비 

In [44]:
# 예시 데이터 로드 (실제 경로를 사용자의 데이터 파일 경로로 바꿔야 함)
df = pd.read_excel('./data/문자클래스데이터.xlsx')

df = df[["Message Body", "label"]]

df

,Message Body,label
0,"[Web발신]\n[KB국민카드]장우진님 08/14 카드대금 561,229원(08/0...",납부 예정
1,"[Web발신]\n[KB국민카드]장우진님 09/13 결제금액 20,000원(연회비 2...",납부 예정
2,"[Web발신]\n[KB국민카드]장우진님 09/14 결제금액 599,554원(연회비 ...",납부 예정
3,"[Web발신]\n[KB국민카드]장우진님 10/16 카드대금 642,364원(09/2...",납부 예정
4,"[Web발신]\n[KB국민카드]장우진님 11/14 카드대금 696,667원(11/0...",납부 예정
...,...,...
304,"[Web발신] MG체크카드(1874) 우*하님 12,500원승인(계좌잔액224,23...",결제 승인
305,"[Web발신] MG체크카드(1874) 우*하님 3,960원승인(계좌잔액236,739...",결제 승인
306,"[Web발신] MG체크카드(1874) 우*하님 40,000원승인(계좌잔액184,23...",결제 승인
307,"[Web발신] SWING 언락패스 이용권(12,900원)이 1일 후 KB국민카드70...",납부 예정


### 라벨 인코딩

In [45]:
label_to_index = {
    '납부 예정': 0,
    '결제 거절': 1,
    '결제 승인': 2,
    '자동 이체': 3,
    '미납': 4,
    '결제 취소': 5,
    '계좌 개설': 6
}

df['label_index'] = df['label'].map(label_to_index)

### 텍스트 데이터 벡터화

In [46]:
# TF-IDF 벡터화
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['Message Body']).toarray()

In [47]:
# TF-IDF 벡터라이저 저장
if not os.path.exists('./vectorizer'):
    os.makedirs('./vectorizer')
vectorizer_path = './vectorizer/tfidf_vectorizer.joblib'
joblib.dump(tfidf, vectorizer_path)

['./vectorizer/tfidf_vectorizer.joblib']

In [48]:
# 라벨
y = df['label_index']

### 학습 데이터 분할

In [49]:
# 학습 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### 모델 구성

In [50]:
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)

### 모델 학습

In [51]:
random_forest.fit(X_train, y_train)
# 예측 및 성능 평가
y_pred = random_forest.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.8709677419354839


### 학습된 모델 테스트

In [52]:
index_to_label = {
    0: '납부 예정',
    1: '결제 거절',
    2: '결제 승인',
    3: '자동 이체',
    4: '미납',
    5: '결제 취소',
    6: '계좌 개설'
}

def get_label_from_index(index):
    return index_to_label.get(index, "Unknown")

In [53]:
def predict_class(text):
    # 텍스트를 TF-IDF 벡터로 변환
    text_vector = tfidf.transform([text]).toarray()
    
    # Random Forest 모델을 사용한 예측 수행
    prediction = random_forest.predict(text_vector)
    
    # 예측된 클래스 인덱스
    predicted_class_index = prediction[0]
    
    return get_label_from_index(predicted_class_index)

In [54]:
# 새로운 텍스트에 대한 클래스 예측
new_text = """[Web발신]
[KB국민카드]장우진님 11/14 카드대금 696,667원(11/03기준) https://m.kbcard.com/z/a1
https://media.hermes.kt.com/data/MEDIA/messagebase/411701d13bde41a3bfe47f311258e345.LT-201105161103520-fgrl.png."""
predicted_class = predict_class(new_text)
print(f"Predicted class for the new text: {predicted_class}")

Predicted class for the new text: 납부 예정


### 학습된 모델 저장

In [55]:
# 모델 저장 디렉토리 확인 및 생성
model_dir = './model'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

model_path = os.path.join(model_dir, 'random_forest_model.pkl')
joblib.dump(random_forest, model_path)

['./model/random_forest_model.pkl']